In [1]:
from bisect import bisect_left
import sys
sys.path.append(r"C:\Users\jarra\Desktop\Masters\Heatwave_Project")
import pandas as pd
import PT13_Functions_For_Masters_New_Test as HW_Func

import Strictly_Functions_Project as SFP
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau


import scipy
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings("ignore")
#RMSE 
from sklearn.metrics import mean_squared_error
from math import sqrt
from datetime import datetime
import seaborn as sns
import math

In [2]:


def Simple_QQ_Regression(Q_step, Historical, Present, Hist_Dates, Pres_Date):
    number = Q_step
    Hist_All = Historical
    Present = Present
    Hist_QQ_Dates_St = Hist_Dates[0]
    Hist_QQ_Dates_En = Hist_Dates[1]

    Pres_QQ_Date_St = Pres_Date[0]
    Pres_QQ_Date_En = Pres_Date[1]
    #^ call in using function
    #^ call in using function

    Historical_30 = Hist_All.loc[Hist_QQ_Dates_St:Hist_QQ_Dates_En].reset_index()
    Present_30 = Present.loc[Pres_QQ_Date_St:Pres_QQ_Date_En].reset_index()



    #Select a nparange value lemgth that goes from Q0 to Q1, and produce the quantiles
    QPRE = Present_30.quantile(np.arange(0,1+number,number)).round(4)
    QPRE = QPRE.rename_axis('Quantile').reset_index()

    QHIS = Historical_30.quantile(np.arange(0,1+number,number)).round(4)
    QHIS = QHIS.rename_axis('Quantile').reset_index()


    #Hist_All = Historical_All.reset_index()
    #This above is the full range of historical data

    #What we will do is append it to max and min values before combiniing with dat
    Hist_Updated_Max = []
    Hist_Updated_Min = []
    Hist_Updated_Date = []



    Hist_All = Hist_All.reset_index()
    #For loop for all dates 
    for i in range(0,len(Hist_All)):
        Hist_Updated_Date.append(Hist_All['date'].loc[i])


        #Now get all the information from the Q-Q data for max and min for each date
        #MAX

        #If data shows a nan value set the updated value to nan
        if (math.isnan(Hist_All['tmax'].loc[i])== True):
            Hist_Updated_Max.append(np.NaN)
        else:
            #Set Temp old 
            Temp_Old = Hist_All['tmax'].loc[i]



            #So now we get the closest value for the max:
            Column = ['tmax']

            #This finds the value where the Q-Hist of the tmax is the minimum it can be for the tmax value presented
            Min_val = np.abs(QHIS[Column] - Temp_Old).min()

            #This finds the quantile*10^5 or by the decimla place you use to find the tmax
            closest_index =  QHIS[np.abs(QHIS[Column]- Temp_Old) == Min_val].stack().idxmin()

            #Now this will use the index to find the Present Vlaue to updayte the historical value to using the index/quantile*10^%
            Hist_Updated_Max.append(QPRE[Column].loc[closest_index[0]].values[0])

        #Now get all the information from the Q-Q data for max and min for each date
        #MIN

        #If data shows a nan value set the updated value to nan
        if (math.isnan(Hist_All['tmin'].loc[i])== True):
            Hist_Updated_Min.append(np.NaN)
        else:
            #Set Temp old 
            Temp_Old = Hist_All['tmin'].loc[i]


            #So now we get the closest value for the max:
            Column = ['tmin']

            #This finds the value where the Q-Hist of the tmax is the minimum it can be for the tmax value presented
            Min_val = np.abs(QHIS[Column] - Temp_Old).min()

            #This finds the quantile*10^5 or by the decimla place you use to find the tmax
            closest_index =  QHIS[np.abs(QHIS[Column]- Temp_Old) == Min_val].stack().idxmin()

            #Now this will use the index to find the Present Vlaue to updayte the historical value to using the index/quantile*10^%
            Hist_Updated_Min.append(QPRE[Column].loc[closest_index[0]].values[0])
    Hist_Updated_Date = pd.DataFrame(Hist_Updated_Date, columns=['date'])
    Hist_Updated_Max = pd.DataFrame(Hist_Updated_Max, columns=['tmax'])
    Hist_Updated_Min = pd.DataFrame(Hist_Updated_Min, columns=['tmin'])


    #Now combine altogether
    Hist_Updated = pd.concat([Hist_Updated_Date, Hist_Updated_Max, Hist_Updated_Min], axis = 1)
    return(Hist_Updated)
        


In [ ]:
Hist_Updated

In [ ]:
#Now putting it all together

#Now lets load thedatasets in
Hist_Max = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\FOR HOMOGENISATION\Tmax_Est_1830_1875.csv")
Hist_Min = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\FOR HOMOGENISATION\Tmin_Est_1830_1875.csv")
Present = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\FOR HOMOGENISATION\ACORN_SAT_1880_2021.csv")

HMax_Col = Hist_Max.columns
HMin_Col = Hist_Min.columns
Present = Present
length_col = len(HMax_Col)

i =1

#Fix the data so its only one trail we are looking at first, this is the columns of the max and min
Hist_Max_Col = Hist_Max[['date',HMax_Col[i]]]
Hist_Min_Col = Hist_Min[['date',HMin_Col[i]]]

#We rename them to they have tmax and tmin
Hist_Max_Col = Hist_Max_Col.rename(columns={HMax_Col[i]:'tmax'})
Hist_Min_Col = Hist_Min_Col.rename(columns={HMin_Col[i]:'tmin'})

#We add the historical datasets together
Hist_Together = pd.concat([Hist_Max_Col['date'], Hist_Max_Col['tmax'],Hist_Min_Col['tmin']],axis = 1)

Hist_Together =Hist_Together.round(4).set_index('date')
Present = Present.set_index('date')

#This is the first trial
Hist_Corrected = Simple_QQ_Regression(0.0001, Hist_Together, Present, ['1846-01-01',"1875-12-31"], ['1880-01-01',"1909-12-31"])

#This is the corrected temps
Hist_Corrected = Hist_Corrected.rename(columns={'tmax':HMax_Col[i]})
Hist_Corrected = Hist_Corrected.rename(columns={'tmin':HMin_Col[i]})

#This is the full list
Hist_Corrected_Dict_Full = Hist_Corrected



for i in range(2,length_col):#length_col):
    print(i)
    #Fix the data so its only one trail we are looking at first
    Hist_Max_Col = Hist_Max[['date',HMax_Col[i]]]
    Hist_Min_Col = Hist_Min[['date',HMin_Col[i]]]


    Hist_Max_Col = Hist_Max_Col.rename(columns={HMax_Col[i]:'tmax'})
    Hist_Min_Col = Hist_Min_Col.rename(columns={HMin_Col[i]:'tmin'})

    Hist_Together = pd.concat([Hist_Max_Col['date'], Hist_Max_Col['tmax'],Hist_Min_Col['tmin']],axis = 1)
    Hist_Together =Hist_Together.round(4).set_index('date')

    Hist_Corrected = Simple_QQ_Regression(0.0001, Hist_Together, Present, ['1846-01-01',"1875-12-31"], ['1880-01-01',"1909-12-31"])
    
    Hist_Corrected = Hist_Corrected.rename(columns={'tmax':HMax_Col[i]})
    Hist_Corrected = Hist_Corrected.rename(columns={'tmin':HMin_Col[i]})
    
    Hist_Corrected_Dict_Full = pd.merge(Hist_Corrected_Dict_Full,Hist_Corrected,on = 'date')



Hist_Corrected_Dict_Full.to_csv(r"C:\Users\jarra\Desktop\Perth_1830_1875_Corrected.csv")



2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [4]:
#Now lets load thedatasets in
Hist_Max = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\FOR HOMOGENISATION\Tmax_Est_1830_1875.csv")
Hist_Min = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\FOR HOMOGENISATION\Tmin_Est_1830_1875.csv")
Present = pd.read_csv(r"E:\LIBRARY\UNIVERSITY\Masters Research\Python\Data\FOR HOMOGENISATION\ACORN_SAT_1880_2021.csv")

HMax_Col = Hist_Max.columns
HMin_Col = Hist_Min.columns
Present = Present
length_col = len(HMin_Col)


In [5]:
length_col

1008